In [2]:
import random
import spacy
import nltk
from goose3 import Goose
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import scrolledtext
from PIL import Image, ImageTk
import io


In [18]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\muril\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [9]:
# Inicializando o spaCy com o modelo de português
nlp = spacy.load('pt_core_news_sm')

# Definindo palavras de boas-vindas
welcome_words_input = ['olá', 'oi', 'hey', 'bom dia', 'boa tarde', 'boa noite']
welcome_words_output = [
    'Olá! Como posso ajudar você sobre sonic hoje?',
    'Olá! Fique à vontade para perguntar sobre sonic.',
    'Oi! Em que posso ajudar com relação à sonic?'
]

In [16]:
# Função para mensagens de boas-vindas
def welcome_message(text):
    for word in text.lower().split():
        if word in welcome_words_input:
            return random.choice(welcome_words_output)
    return None

# Função de pré-processamento de texto
def preprocessing(sentence):
    sentence = sentence.lower()
    tokens = [token.text for token in nlp(sentence) if not (token.is_stop or token.like_num
                                                              or token.is_punct or token.is_space
                                                              or len(token) == 1)]
    return ' '.join(tokens)

In [19]:
# Coletando e processando as sentenças
original_sentences = []

g = Goose()
article = g.extract(url='https://pt.wikipedia.org/wiki/Sonic_the_Hedgehog')

sentences = nltk.sent_tokenize(article.cleaned_text)
original_sentences.extend(sentences)

In [20]:
# Pré-processamento das sentenças
cleaned_sentences = [preprocessing(sentence) for sentence in original_sentences]

# Função para responder às perguntas do usuário
def answer(user_text, threshold=0.2):
    user_text_clean = preprocessing(user_text)
    all_sentences = cleaned_sentences + [user_text_clean]
    
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(all_sentences)
    
    similarity = cosine_similarity(tfidf[-1], tfidf)
    similarity_scores = similarity.flatten()
    
    # Encontrar a sentença com maior similaridade, excluindo a própria
    highest_idx = similarity_scores.argsort()[-2]
    highest_score = similarity_scores[highest_idx]
    
    if highest_score < threshold:
        return 'Desculpe, não encontrei uma resposta adequada para sua pergunta.'
    else:
        return original_sentences[highest_idx]

# Função para gerar a nuvem de palavras
def generate_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    return wordcloud

In [24]:

# Preparando a base de conhecimento para a nuvem de palavras
full_text = ' '.join(cleaned_sentences)
wordcloud = generate_wordcloud(full_text)

# Salvando a nuvem de palavras em memória
image_stream = io.BytesIO()
wordcloud.to_image().save(image_stream, format='PNG')
image_stream.seek(0)
wordcloud_image = Image.open(image_stream)

# Função do Chatbot para a interface
def send_message():
    user_input = user_entry.get()
    if user_input.lower() in ['sair', 'quit', 'exit']:
        chat_window.config(state=NORMAL)
        chat_window.insert(END, "Chatbot: Até logo! Bom apetite!\n")
        chat_window.config(state=DISABLED)
        root.after(2000, root.destroy)
        return
    if user_input.strip() == '':
        return
    chat_window.config(state=NORMAL)
    chat_window.insert(END, f"Você: {user_input}\n")
    user_entry.delete(0, END)
    
    welcome = welcome_message(user_input)
    if welcome:
        response = welcome
    else:
        response = answer(user_input)
    chat_window.insert(END, f"Chatbot: {response}\n")
    chat_window.yview(END)
    chat_window.config(state=DISABLED)

# Inicializando a janela do Tkinter
root = Tk()
root.title("Chatbot de Culinária Brasileira")
root.geometry("800x600")
root.resizable(False, False)

# Frame para o chat
chat_frame = Frame(root)
chat_frame.pack(pady=10)

# Área de texto para exibir o chat
chat_window = scrolledtext.ScrolledText(chat_frame, wrap=WORD, width=80, height=20, state=DISABLED)
chat_window.pack()

# Frame para a entrada do usuário
input_frame = Frame(root)
input_frame.pack(pady=10)

# Entrada de texto para o usuário
user_entry = Entry(input_frame, width=70)
user_entry.pack(side='left', padx=10)

# Botão para enviar a mensagem
send_button = Button(input_frame, text="Enviar", command=send_message)
send_button.pack(side='left')

# Frame para a nuvem de palavras
wordcloud_frame = Frame(root)
wordcloud_frame.pack(pady=10)

# Label para a nuvem de palavras
wordcloud_label = Label(wordcloud_frame)
wordcloud_label.pack()

# Função para exibir a nuvem de palavras
# def display_wordcloud():
#     resized_image = wordcloud_image.resize((750, 375), Image.Resampling.LANCZOS)
#     tk_image = ImageTk.PhotoImage(resized_image)
#     wordcloud_label.configure(image=tk_image)
#     wordcloud_label.image = tk_image

# Exibindo a nuvem de palavras ao iniciar
# display_wordcloud()

# Iniciando o Chatbot
def start_chatbot():
    chat_window.config(state=NORMAL)
    chat_window.insert(END, "Chatbot: Olá! Eu sou um chatbot. Faça sua pergunta ou digite 'sair' para encerrar.\n")
    chat_window.config(state=DISABLED)

start_chatbot()

# Bind Enter key to send message
root.bind('<Return>', lambda event: send_message())

# Executando a interface
root.mainloop()
